In [17]:
import pandas as pd
import numpy as np
from glob import glob

file_name = "transfer_receiving.csv"
df = pd.DataFrame()
df_list = []
for f in glob(file_name):
    print(f)
    sub_df = pd.read_csv(f)
    df_list.append(sub_df)
df = pd.concat(df_list,axis=0)
df = df.loc[df.To.isin(["A1"])]
df = df.rename({"Trnf#":"TransferNumber"},axis=1)
df.groupby(["TransferNumber","To"]).agg({"O Qty":"sum"})

transfer_receiving.csv


,,O Qty
TransferNumber,To,
2507,A1,7956
2508,A1,8365
2509,A1,1950
2510,A1,3613
2511,A1,1551
2512,A1,2459
2715,A1,7061
2753,A1,6082
2754,A1,4988


In [15]:
df.loc[df.TransferNumber==2888]

,TransferNumber,Line,Priority,Sts,Style,Clr,Style Description,Color Description,Size Type,Size,...,Style Color FOB,BOL #,Division,Division Description,Label,Label Description,Category,Category Description,NAFTA Compliant,UPC Code


In [5]:
file_name = "transfer_receiving.csv"
df = pd.DataFrame()
df_list = []
for f in glob(file_name):
    print(f)
    sub_df = pd.read_csv(f)
    df_list.append(sub_df)
df = pd.concat(df_list,axis=0)
df = df.loc[~df.To.isin(["A1","NR"])]
df = df.rename({"Trnf#":"TransferNumber"},axis=1)
df.groupby(["TransferNumber","To"]).agg({"O Qty":"sum"})

transfer_receiving.csv


,,O Qty
TransferNumber,To,
2829,V5,440
2833,VM,276
2834,V6,99
2837,VR,2191
2838,V5,1569
2840,V6,853
2841,V3,795
2844,V3,69
2846,VR,805


In [11]:
order_num = 2840
file_name = str(order_num) + ".csv"

In [12]:
def change_sizes(string):
    if string == "S":
        return "SML"
    if string == "XS":
        return "XSM"
    if string == "XXS":
        return "XXS"
    if string == "M":
        return "MED"
    if string == "L":
        return "LRG"
    if string == "XL":
        return "XLG"
    if string == "XXL":
        return "XXL"
    else:
        return string
def apply_numeric_size(string):
    if string.isnumeric():
        string = string.zfill(3)
    return string
df["Size"] = df.Size.apply(apply_numeric_size)
df["Size"] = df.Size.apply(change_sizes)
df["SKU_full"] = df["Style"] + df["Clr"] + df["Size"]

In [13]:
purchase_order = pd.read_csv(file_name)
purchase_order.groupby("Barcode").agg({"Qty Ordered":"sum"})
df_2821= df.loc[df["TransferNumber"] == order_num]
df_2821["UPC Code"] = df_2821["UPC Code"].astype("int").astype("str")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [14]:
check_0 = purchase_order.groupby("SKU").agg({"Qty Ordered":"sum"})["Qty Ordered"] - df_2821.groupby("SKU_full").agg({"Shp Qty":"sum"})["Shp Qty"]
check_0.loc[check_0 != 0].index

Index(['VW339HKEMED'], dtype='object')

In [15]:
for item_id in check_0.loc[check_0 != 0].index:
    print(item_id)
    print("Received")
    print(purchase_order.loc[purchase_order.SKU == item_id]["Qty Ordered"])
    print("On FULL circle")
    print(df_2821.loc[df_2821.SKU_full == item_id]["Shp Qty"])
    print("=================")


VW339HKEMED
Received
217    4
Name: Qty Ordered, dtype: int64
On FULL circle
Series([], Name: Shp Qty, dtype: int64)


In [16]:
item_id = "VW339HTGSML"
print("Received")
print(purchase_order.loc[purchase_order.SKU == item_id]["Qty Ordered"])
print("On FULL circle")
df_2821.loc[df_2821.SKU_full == item_id]["Shp Qty"]


Received
Series([], Name: Qty Ordered, dtype: int64)
On FULL circle


Series([], Name: Shp Qty, dtype: int64)

V185CCLMED     1.0
VW158BLKMED    NaN
VW303BLKSML    1.0
dtype: float64